In [2]:
# from google.colab import files
# uploaded = files.upload()

In [3]:
#Data Prepping

import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("../data/ai4i2020.csv")
df.head()

# How many rows & colums + NaN check
print("Initial shape:", df.shape)
print("\nNaN check:\n", df.isna().sum())

# Drop any NaN rows if exist
df.dropna(inplace=True)
print("\nShape after removing NaN rows:", df.shape)

# Encode product quality labels (L, M, H) numerically as 0, 1 & 2
if 'Type' in df.columns:
    df['Type'] = df['Type'].map({'L': 0, 'M': 1, 'H': 2})

# Select continuous numerical columns
continuous_cols = ['Air temperature [K]', 'Process temperature [K]',
                   'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

# Apply Z-score scaling
scaler_z = StandardScaler()
df_z_scaled = df.copy()
df_z_scaled[continuous_cols] = scaler_z.fit_transform(df[continuous_cols])

# Final output
print("\nZ-score scaled sample:")
print(df_z_scaled.head())


Initial shape: (10000, 14)

NaN check:
 UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

Shape after removing NaN rows: (10000, 14)

Z-score scaled sample:
   UDI Product ID  Type  Air temperature [K]  Process temperature [K]  \
0    1     M14860     1            -0.952389                -0.947360   
1    2     L47181     0            -0.902393                -0.879959   
2    3     L47182     0            -0.952389                -1.014761   
3    4     L47183     0            -0.902393                -0.947360   
4    5     L47184     0            -0.902393                -0.879959   

   Rotational speed [rpm]  Torque 